In [ ]:
import pandas as pd
import math
import numpy as np
import pickle

In [ ]:
sabiork_kcat_df = pd.read_csv("./sabiork_data_cache/sabiork_delta_kcat_df.csv")
sabiork_km_df = pd.read_csv("./sabiork_data_cache/sabiork_delta_km_df.csv")
brenda_kcat_df = pd.read_csv("./brenda_data_cache/brenda_delta_kcat_df.csv")
brenda_km_df = pd.read_csv("./brenda_data_cache/brenda_delta_km_df.csv")
len(brenda_kcat_df),len(brenda_km_df)

#### (a) Remove duplicate entries between the two databases.

In [ ]:
def merge_brenda_sabiork(brenda_df,sabiork_df,target):
    standard_col_name = ['EcNumber','Organism','Substrate','UniprotId','sequence','mutant','Temperature','pH',
                         f'wt_{target}_log10',f'mut_{target}_log10',f'delta_{target}_log10']
    brenda_df = brenda_df[standard_col_name].copy()
    sabiork_df = sabiork_df[standard_col_name].copy()
    brenda_df['Temperature'] = brenda_df['Temperature'].astype(str)
    brenda_df['pH'] = brenda_df['pH'].astype(str)
    brenda_df['mutant'] = brenda_df['mutant'].str.upper()
    sabiork_df['Temperature'] = sabiork_df['Temperature'].astype(str)
    sabiork_df['pH'] = sabiork_df['pH'].astype(str)
    sabiork_df['mutant'] = sabiork_df['mutant'].str.upper()

    remove_loc = []
    for i,row in brenda_df.iterrows():
        conditions = {'EcNumber':row['EcNumber'],'Organism':row['Organism'],'Substrate':row['Substrate'],'UniprotId':row['UniprotId'],'sequence':row['sequence'],
                    'mutant':row['mutant'],'Temperature':row['Temperature'],'pH':row['pH'],f'wt_{target}_log10':row[f'wt_{target}_log10'],f'mut_{target}_log10':row[f'mut_{target}_log10'],
                    f'delta_{target}_log10':row[f'delta_{target}_log10']}
        result_indexs = sorted(sabiork_df[(sabiork_df[list(conditions)] == pd.Series(conditions)).all(axis=1)].index.tolist())
        remove_loc+=result_indexs
    sabiork_df = sabiork_df.iloc[[i for i in range(len(sabiork_df)) if i not in remove_loc]].copy()
    
    merge_df = pd.concat([brenda_df,sabiork_df],ignore_index=True)
    merge_df = merge_df[(merge_df[f'wt_{target}_log10']>-5) & (merge_df[f'wt_{target}_log10']<5)].copy()
    merge_df = merge_df[(merge_df[f'mut_{target}_log10']>-5) & (merge_df[f'mut_{target}_log10']<5)].copy()
    merge_df.reset_index(drop=True, inplace=True)
    return merge_df
merge_kcat_df = merge_brenda_sabiork(brenda_kcat_df,sabiork_kcat_df,'kcat')
merge_km_df = merge_brenda_sabiork(brenda_km_df,sabiork_km_df,'km')
len(merge_kcat_df),len(merge_km_df)

#### (b) Merge entries from the same enzymatic reaction but under different experimental conditions.
We first grouped mutation effect pairs with the same EC number, enzyme and substrate but different experimental conditions or literature sources. Then, we excluded data points likely measured under non-optimal conditions. Finally, we averaged the remaining data points within each group to represent the overall mutation effects across assay conditions.

In [ ]:
def merge_df(df,target):
    def process_group(group):
        if target =='km':
            upper = group[f'wt_{target}_log10'].min() + 2  # log(100)
            filtered_group = group[group[f'wt_{target}_log10'] <= upper]
        
        elif target =='kcat':
            bottom = group[f'wt_{target}_log10'].max() - 2  # log(100)
            filtered_group = group[group[f'wt_{target}_log10'] >= bottom]
        
        avg_delta_k = filtered_group[f'delta_{target}_log10'].mean()
        wt_k_log10 = filtered_group[f'wt_{target}_log10'].mean()
        mut_k_log10 = filtered_group[f'mut_{target}_log10'].mean()

        return pd.Series({f'delta_{target}_log10': avg_delta_k, f'wt_{target}_log10': wt_k_log10,f'mut_{target}_log10':mut_k_log10})

    df = df.groupby(['EcNumber', 'Organism','Substrate','UniprotId','sequence','mutant']).apply(process_group).reset_index()
    return df

delta_kcat_df = merge_df(merge_kcat_df,'kcat')
delta_km_df = merge_df(merge_km_df,'km')
len(delta_kcat_df),len(delta_km_df)

In [ ]:
def get_mutant_sequence(df):
    mut_seqs = []
    for i,row in df.iterrows():
        muts = row['mutant']
        mut_seq = row['sequence']
        for mut in muts.split(","):
            loc = int(mut[1:-1])
            mut_seq=mut_seq[:loc]+mut[-1]+mut_seq[loc+1:]
        mut_seqs.append(mut_seq)
    return mut_seqs
    
    
delta_kcat_df['wildtype_sequence']=delta_kcat_df['sequence']
delta_km_df['wildtype_sequence']=delta_km_df['sequence']
delta_kcat_df['mutant_sequence']=get_mutant_sequence(delta_kcat_df)
delta_km_df['mutant_sequence']=get_mutant_sequence(delta_km_df)

delta_kcat_df = delta_kcat_df[['EcNumber','Organism','Substrate','UniprotId','mutant','delta_kcat_log10','wildtype_sequence','mutant_sequence']]
delta_km_df = delta_km_df[['EcNumber','Organism','Substrate','UniprotId','mutant','delta_km_log10','wildtype_sequence','mutant_sequence']]
delta_kcat_df.to_csv("./delta_kcat.csv",index=False)
delta_km_df.to_csv("./delta_km.csv",index=False)